In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.10)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 2.02 sec - Loss 0.060906 - ACC 62.10% - ACC Mean 62.10% - AUC 66.76% - AUC Mean 66.76% - Deter 000
Ite 00050 - 0.94 sec - Loss 0.012443 - ACC 61.80% - ACC Mean 61.23% - AUC 65.98% - AUC Mean 65.85% - Deter 048
Ite 00052 - 0.94 sec - Loss 0.012063 - ACC 61.70% - ACC Mean 61.21% - AUC 66.24% - AUC Mean 65.81% - Deter 050
Early stop ite 52, rollback to correction of ite 2, whith acc of 63.75% and auc of 68.11%
OUR METHOD RUN 1/10 - acc: 63.75% +- 0.0% - auc: 68.11% +- 0.0%
Ite 00000 - 0.93 sec - Loss 0.140771 - ACC 62.40% - ACC Mean 62.40% - AUC 66.40% - AUC Mean 66.40% - Deter 000
Ite 00050 - 1.01 sec - Loss 0.053657 - ACC 60.75% - ACC Mean 60.87% - AUC 65.20% - AUC Mean 65.56% - Deter 002
Ite 00098 - 0.93 sec - Loss 0.035558 - ACC 61.65% - ACC Mean 61.04% - AUC 66.48% - AUC Mean 65.89% - Deter 050
Early stop ite 98, rollback to correction of ite 48, whith acc of 62.65% and auc of 68.3%
OUR METHOD RUN 2/10 - acc: 62.65% +- 0.0% - auc: 68.3% +- 0.0%
Ite 00000 - 0.96 sec - Loss

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 61.2% +- 0.0% - auc: 65.47% +- 0.0%
MEAN - acc: 63.4% +- 0.0% - auc: 68.55% +- 0.0%
KNN - acc: 59.95% +- 0.0% - auc: 63.25% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:40<00:00, 122.38it/s]



GAIN RUN 1/10 - acc: 62.15% +- 2.82% - auc: 66.4% +- 3.05%



100%|██████████| 5000/5000 [00:41<00:00, 121.01it/s]



GAIN RUN 2/10 - acc: 60.9% +- 2.91% - auc: 65.51% +- 2.98%



100%|██████████| 5000/5000 [00:42<00:00, 118.12it/s]


...


GAIN RUN 8/10 - acc: 61.85% +- 2.63% - auc: 66.34% +- 2.21%



100%|██████████| 5000/5000 [00:41<00:00, 120.90it/s]



GAIN RUN 9/10 - acc: 60.35% +- 2.76% - auc: 65.25% +- 2.53%



100%|██████████| 5000/5000 [00:45<00:00, 110.68it/s]



GAIN RUN 10/10 - acc: 61.5% +- 0.72% - auc: 65.81% +- 2.31%

GAIN GLOBAL - acc: 61.76% +- 0.8% - auc: 66.13% +- 0.56%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 58.8% +- 0.0% - auc: 62.58% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 60.8% +- 1.87% - auc: 65.8% +- 1.49%

Iteration: 0
Iteration: 1

MISSFOREST RUN 2/10 - acc: 59.35% +- 1.25% - auc: 63.35% +- 2.19%

Iteration: 0
Iteration: 1

MISSFOREST RUN 3/10 - acc: 60.45% +- 1.35% - auc: 63.95% +- 0.68%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 58.5% +- 2.47% - auc: 63.38% +- 3.45%

Iteration: 0
Iteration: 1

MISSFOREST RUN 5/10 - acc: 61.35% +- 4.56% - auc: 65.37% +- 4.61%

Iteration: 0
Iteration: 1

MISSFOREST RUN 6/10 - acc: 61.95% +- 2.34% - auc: 65.97% +- 2.15%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 7/10 - acc: 61.8% +- 2.59% - auc: 65.41% +- 3.73%

Iteration: 0
Iteration: 1

MISSFOREST RUN 8/10 - acc: 62.2% +- 2.6% - auc: 67.01% +- 3.33%

Iteration: 0
Iteration: 1

MISSFOREST RUN 9/10 - acc: 62.2% +- 2.59% - auc: 67.16% +- 2.61%

Iteration: 0
Iteration: 1
Iteration: 2

MISSFOREST RUN 10/10 - acc: 61.7% +- 2.48% - auc: 65.68% +- 3.18%


MISSFOREST GLOBAL - acc: 61.03% +- 1.2% 

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 60.9% +- 2.15% - auc: 66.3% +- 2.22%
SOFTIMPUTE RUN 2/10 - acc: 61.15% +- 1.45% - auc: 66.23% +- 1.69%
SOFTIMPUTE RUN 3/10 - acc: 60.4% +- 1.68% - auc: 64.72% +- 2.35%
SOFTIMPUTE RUN 4/10 - acc: 60.8% +- 2.87% - auc: 64.98% +- 2.68%
SOFTIMPUTE RUN 5/10 - acc: 61.75% +- 0.61% - auc: 65.94% +- 2.23%
SOFTIMPUTE RUN 6/10 - acc: 60.8% +- 1.75% - auc: 66.08% +- 1.15%
SOFTIMPUTE RUN 7/10 - acc: 60.7% +- 2.8% - auc: 66.2% +- 4.22%
SOFTIMPUTE RUN 8/10 - acc: 59.8% +- 2.48% - auc: 64.42% +- 3.57%
SOFTIMPUTE RUN 9/10 - acc: 61.25% +- 3.95% - auc: 65.67% +- 3.57%
SOFTIMPUTE RUN 10/10 - acc: 61.15% +- 0.68% - auc: 66.05% +- 1.9%

SOFTIMPUTE GLOBAL - acc: 60.87% +- 0.5% - auc: 65.66% +- 0.66%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1106

SINKHORN RUN 1/10 - acc: 59.7% +- 1.51% - auc: 63.16% +- 1.55%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1005

SINKHORN RUN 2/10 - acc: 58.9% +- 1.83% - auc: 63.18% +- 2.51%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.0833

SINKHORN RUN 3/10 - acc: 59.75% +- 2.46% - auc: 63.79% +- 3.34%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.2575

SINKHORN RUN 4/10 - acc: 60.0% +- 2.49% - auc: 63.56% +- 2.46%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1348

SINKHORN RUN 5/10 - acc: 60.25% +- 2.49% - auc: 64.36% +- 3.02%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1262

SINKHORN RUN 6/10 - acc: 60.15% +- 1.63% - auc: 64.92% +- 2.76%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.0659

SINKHORN RUN 7/10 - acc: 59.05% +- 2.82% - auc: 63.11% +- 2.16%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 2.1725

SINKHORN RUN 8/10 - acc: 61.8% +- 2.69% - auc: 66.26

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 60.45% +- 1.84% - auc: 65.5% +- 2.41%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 61.2% +- 2.08% - auc: 65.47% +- 1.79%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 60.85% +- 1.84% - auc: 65.39% +- 1.89%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 61.25% +- 2.94% - auc: 65.65% +- 3.12%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 59.25% +- 2.3% - auc: 63.77% +- 2.3%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 59.85% +- 2.66% - auc: 64.64% +- 2.87%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 60.1% +- 3.06% - auc: 64.03% +- 3.89%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 61.8% +- 2.23% - auc: 67.08% +- 3.65%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 62

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is even with MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bet